### Notes
- use cath as becnhmarking
- eg with pdb structure u get 85%, alphafold 84%, that's good 
- something like an upper limit
- use ./SWORD2.py -p 1jx4 -o results instead of downloading pdb files

In [2]:
from pprint import pprint
import numpy as np
import pandas as pd

In [3]:
def update_dict(data, code, chain, domain, boundaries):
    data[code] = data.get(code, {})
    if chain not in data[code].keys():
        data[code][chain] = {}
    data[code][chain][str(int(domain))] = boundaries
    return data


def get_cath(lines_range = None, file="../data/cath/cath_domain_boundaries.txt", verb=False):
    with open(file, "r") as f:
        lines = f.readlines() if lines_range is None else f.readlines()[lines_range[0]:lines_range[1]]
        lines = map(lambda x : x.strip(), lines)
        big = 0
        data = {}
        for line in lines:
            name, boundaries = line.split("\t")
            code = name[:4]
            chain = name[4]
            domain = name[5:]
            data = update_dict(data, code, chain, domain, boundaries)
    verb and pprint(data)
    return data


def get_sword2(code, chain, verb=False):
    
    file = f"../data/sword2/SWORD2/results/{code}_{chain.upper()}/sword.txt"
    with open(file, "r") as f:
        data = {}
        lines = f.readlines()
        option = 0
        for i, line in enumerate(lines):
            lines[i] = "".join([c for c in line if c not in ["\n",'']])
            if line != "\n":
                if not line.startswith(("PDB:", "#D", "A")):
                    res = lines[i].split("|")
                    boundaries = res[2]
                    domains = boundaries.strip().split(" ")
                    data[f"option{option}"] = {}
                    for j in range(len(domains)):
                        data[f"option{option}"][str(j+1)] = domains[j]
                    option += 1
    verb and pprint(data)
    return data


def cath_to_file(cath):
    with open('list.txt', 'w') as f:
        for key in cath.keys():
            f.write(key + ',')

In [4]:
cath = get_cath()
cath_to_file(cath)

In [7]:
# cath['1a0h']
cath

{'101m': {'A': {'0': '1-154'}},
 '102l': {'A': {'0': '1-163'}},
 '102m': {'A': {'0': '1-154'}},
 '103l': {'A': {'0': '1-165'}},
 '103m': {'A': {'0': '1-154'}},
 '104l': {'A': {'0': '1-164'}, 'B': {'0': '1-164'}},
 '104m': {'A': {'0': '1-153'}},
 '105m': {'A': {'0': '1-153'}},
 '106m': {'A': {'0': '1-154'}},
 '107l': {'A': {'0': '1-162'}},
 '107m': {'A': {'0': '1-154'}},
 '108l': {'A': {'0': '1-162'}},
 '108m': {'A': {'0': '1-154'}},
 '109l': {'A': {'0': '1-162'}},
 '109m': {'A': {'0': '1-154'}},
 '10gs': {'A': {'1': '2-78,187-208', '2': '79-186'},
  'B': {'1': '2-78,187-208', '2': '79-186'}},
 '10mh': {'A': {'1': '1-186,285-327', '2': '187-284'}},
 '110l': {'A': {'0': '1-162'}},
 '110m': {'A': {'0': '1-154'}},
 '111l': {'A': {'0': '1-162'}},
 '111m': {'A': {'0': '1-154'}},
 '112l': {'A': {'0': '1-162'}},
 '112m': {'A': {'0': '1-154'}},
 '113l': {'A': {'0': '1-162'}},
 '114l': {'A': {'0': '1-162'}},
 '115l': {'A': {'0': '1-162'}},
 '116l': {'A': {'0': '1-162'},
  'B': {'0': '1-164'},
  

In [5]:
def accuracy(d1, d2):
    # Accuracy = no of correct predictions / sequence size (x) -- correct later     
    seq_size = max(sequence_size(d1), sequence_size(d2))
    lst1 = domains_to_seq(d1, seq_size)
    lst2 = domains_to_seq(d2, seq_size)
    results = list(map(lambda x : x[0] == x[1], zip(lst1, lst2)))
    
    overlap = 0
    diff = 0
    for elt in results:
        if elt == True:
            overlap += 1
        else:
            diff += 1
            
    result = (overlap / (overlap + diff)) * 100
    # accuracy as percentage
    return float("{:.2f}".format(result)) 



def sequence_size(d):
    size = 0
    for value in d.values():
        value = int(value.split('-')[-1])
        if value > size:
            size = value
    return size


def compare(pdb, chain, f=accuracy, verb=True):
    accs = []
    for option in get_sword2(pdb, chain).keys():
        x = cath[pdb][chain]
        y = get_sword2(pdb, chain)[option]
        acc = f(x,y)
        verb and print(f"x: {x}\ny: {y}")
        verb and print(f"{option} accuracy: {acc}")
        accs.append((option, acc))
    return accs
    
    
def domains_to_seq(d, seq_size):
    no_of_domains = len(d.keys())
    lst = [0] * seq_size
    for i in range(1, no_of_domains + 1):
        domain = d.get(str(i), '0-0')
        for n in range(int(domain.split('-')[0])-1, int(domain.split('-')[-1])):
            lst[n] = i
    return lst

In [9]:
# pdbs = ['1a0h','1abe', '1acc', '1an9', '1b4v']
# pdb = '1an9'
# chain = 'A'
# for option in get_sword2(pdb, chain).keys():
#         x = cath[pdb][chain]
#         y = get_sword2(pdb, chain)[option]
#     #     print(f"x: {x}\ny: {y}")
#         acc = accuracy(y,x)
#         print(f"{option} accuracy: {acc}%")
    

In [6]:
# pdbs = ['1a0h','1abe', '1acc', '1an9', '1b4v']
pdbs = ['1a0h', '1an9', '1b4v']
chain = 'A'

In [11]:
# def casp6_acc(x, y):
    
#     target = [0] * len(x.keys())
#     predictions = [0] * len(y.keys())
    
#     seq_size = max(sequence_size(x), sequence_size(y))
#     lst_x = domains_to_seq(x, seq_size)
#     lst_y = domains_to_seq(y, seq_size)
    
#     for (t, p) in zip(lst_x, lst_y):
#         if t
            
#     result = 0
#     # accuracy as percentage
#     return float("{:.4f}".format(result)) * 100

In [10]:
def casp6_acc(x, y):
    
#     table = np.zeros((len(x.keys()) + 1, len(y.keys()) + 1))
    x_size = len(x.keys())
    y_size = len(y.keys())
    table = np.zeros((x_size, y_size))

    seq_size = max(sequence_size(x), sequence_size(y))
    lst_x = domains_to_seq(x, seq_size)
    lst_y = domains_to_seq(y, seq_size)
    for (t, p) in zip(lst_x, lst_y):
        if t == 0 or p == 0:
            continue
        table[t-1, p-1] += 1
            
    row_score = [0] * y_size
    col_score = [0] * x_size
    
    for i in range(y_size):
        for j in range(x_size):
            if i == j:
                row_score[i] += table[j, i]
                col_score[j] += table[j, i]
            else:
                row_score[i] -= table[j, i]
                col_score[j] -= table[j, i]
        
    
    score = (np.sum(row_score) + np.sum(col_score)) / 2
    normalized_score = score / sequence_size(x)
    # accuracy as percentage
#     print("table")
#     print(table)
#     print("row")
#     print(row_score)
#     print("col")
#     print(col_score)
#     print("score")
#     print(score)
#     print("norm score")
#     print(normalized_score)
    return float("{:.4f}".format(normalized_score)) * 100


In [11]:
print("CASP6\n")
for pdb in pdbs:
    score = compare(pdb, chain, casp6_acc, False)
    print(pdb)
    for (opt, y) in score:
        print(f"{opt}:\n{y}")

CASP6

1a0h
option0:
13.84
option1:
-32.7
option2:
1.26
option3:
84.28
option4:
71.7
1an9
option0:
74.63
option1:
-41.0
option2:
-35.69
option3:
-9.139999999999999
option4:
-6.1899999999999995
option5:
34.510000000000005
1b4v
option0:
-15.8
option1:
9.0
option2:
9.0
option3:
53.400000000000006
option4:
24.6


In [14]:
print("Accuracy\n")
for pdb in pdbs:
    score = compare(pdb, chain, accuracy, False)
    print(pdb)
    for (opt, y) in score:
        print(f"{opt}: {y}")

Accuracy

1a0h
option0: 55.97
option1: 32.7
option2: 49.69
option3: 91.19
option4: 84.91
1an9
option0: 87.06
option1: 29.41
option2: 32.06
option3: 45.29
option4: 46.76
option5: 67.06
1b4v
option0: 41.6
option1: 54.0
option2: 54.0
option3: 76.2
option4: 61.8


In [15]:
{1 : 1-150, 2: 151-200}

{1: -149, 2: -49}

In [16]:
cath

{'101m': {'A': {'0': '1-154'}},
 '102l': {'A': {'0': '1-163'}},
 '102m': {'A': {'0': '1-154'}},
 '103l': {'A': {'0': '1-165'}},
 '103m': {'A': {'0': '1-154'}},
 '104l': {'A': {'0': '1-164'}, 'B': {'0': '1-164'}},
 '104m': {'A': {'0': '1-153'}},
 '105m': {'A': {'0': '1-153'}},
 '106m': {'A': {'0': '1-154'}},
 '107l': {'A': {'0': '1-162'}},
 '107m': {'A': {'0': '1-154'}},
 '108l': {'A': {'0': '1-162'}},
 '108m': {'A': {'0': '1-154'}},
 '109l': {'A': {'0': '1-162'}},
 '109m': {'A': {'0': '1-154'}},
 '10gs': {'A': {'1': '2-78,187-208', '2': '79-186'},
  'B': {'1': '2-78,187-208', '2': '79-186'}},
 '10mh': {'A': {'1': '1-186,285-327', '2': '187-284'}},
 '110l': {'A': {'0': '1-162'}},
 '110m': {'A': {'0': '1-154'}},
 '111l': {'A': {'0': '1-162'}},
 '111m': {'A': {'0': '1-154'}},
 '112l': {'A': {'0': '1-162'}},
 '112m': {'A': {'0': '1-154'}},
 '113l': {'A': {'0': '1-162'}},
 '114l': {'A': {'0': '1-162'}},
 '115l': {'A': {'0': '1-162'}},
 '116l': {'A': {'0': '1-162'},
  'B': {'0': '1-164'},
  

In [19]:
def cath_to_csv(cath):
    df = pd.DataFrame(columns=['Codes', 'Sequence', 'Domains'])
    codes = []
    sequence = []
    domains = []
    for key, val in cath.items():
        code = key
        

    return df

cath_to_csv(cath)

,codes
0,101m
1,102l
2,102m
3,103l
4,103m
...,...
137898,9rsa
137899,9rub
137900,9wga
137901,9xia


In [35]:
from Bio import SeqIO
obj1 = SeqIO.parse("../data/uniprot_sprot.dat", "swiss")
for record in obj1:
    print("%s %i" % (record.id, len(record)))
    print(record.seq)
    break

# obj = SeqIO.index("../data/uniprot_sprot.dat", "swiss")
# obj



Q6GZX4 256
MAFSAEDVLKEYDRRRRMEALLLSLYYPNDRKLLDYKEWSPPRVQVECPKAPVEWNNPPSEKGLIVGHFSGIKYKGEKAQASEVDVNKMCCWVSKFKDAMRRYQGIQTCKIPGKVLSDLDAKIKAYNLTVEGVEGFVRYSRVTKQHVAAFLKELRHSKQYENVNLIHYILTDKRVDIQHLEKDLVKDFKALVESAHRMRQGHMINVKYILYQLLKKHGHGPDGPDILTVKTGSKGVLYDDSFRKIYTDLGWKFTPL


In [37]:
x = obj.get('Q6GZX4')
x


SeqRecord(seq=Seq('MAFSAEDVLKEYDRRRRMEALLLSLYYPNDRKLLDYKEWSPPRVQVECPKAPVE...TPL'), id='Q6GZX4', name='001R_FRG3G', description='RecName: Full=Putative transcription factor 001R;', dbxrefs=['EMBL:AY548484', 'RefSeq:YP_031579.1', 'SwissPalm:Q6GZX4', 'GeneID:2947773', 'KEGG:vg:2947773', 'Proteomes:UP000008770', 'GO:GO:0046782', 'InterPro:IPR007031', 'Pfam:PF04947'])